In [1]:
## Importing packages

# This R environment comes with all of CRAN and many other helpful packages preinstalled.
# You can see which packages are installed by checking out the kaggle/rstats docker image: 
# https://github.com/kaggle/docker-rstats

library(tidyverse) # metapackage with lots of helpful functions
library(reshape2)
library(data.table)
library(caret)
library(dplyr)
library(lubridate)
library(gbm)
library(randomForest)
library(adabag)
library(plyr)
set.seed(303)

## Running code

# In a notebook, you can run a single code cell by clicking in the cell and then hitting 
# the blue arrow to the left, or by clicking in the cell and pressing Shift+Enter. In a script, 
# you can run code by highlighting the code you want to run and then clicking the blue arrow
# at the bottom of this window.

## Reading in files

# You can access files from datasets you've added to this kernel in the "../input/" directory.
# You can see the files added to this kernel by running the code below. 

list.files(path = "../input")

train <- fread("../input/biost2080-hw2-new/new_train.csv", header = TRUE)
test <- fread('../input/biost2080-hw2-new/new_test.csv', header = T)

## Saving data

# If you save any files or images, these will be put in the "output" directory. You 
# can see the output directory by committing and running your kernel (using the 
# Commit & Run button) and then checking out the compiled version of your kernel.

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.2.1.9000     ✔ purrr   0.3.3     
✔ tibble  2.1.3          ✔ dplyr   0.8.3     
✔ tidyr   1.0.0          ✔ stringr 1.4.0     
✔ readr   1.3.1          ✔ forcats 0.4.0     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths



Attaching package: ‘data.table’


The following objects are masked from ‘package:reshape2’:

    dcast, melt


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


The following object is masked from ‘package:httr’:

    progress



Attaching 

[1] "biost2080-hw2-new"

In [2]:
head(train)
head(test)

Id,Category,GENDER,FIRST_CAREUNIT,AnionGap,BaseExcess,Basophils,Bicarbonate,BilirubinTotal,CalciumTotal,⋯,Potassium,RDW,RedBloodCells,Sodium,SpecificGravity,UreaNitrogen,WhiteBloodCells,pCO2,pH,pO2
<int>,<int>,<chr>,<chr>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>
2,0,M,CSRU,14,2,0.0,35,0.4,9.0,⋯,5.9,14.2,4.21,152,1.017,25,34.5,52,7.46,278
3,0,F,CSRU,12,0,0.4,28,0.7,9.1,⋯,5.4,14.4,3.38,142,1.048,24,14.2,35,5.00,194
4,0,M,CSRU,12,3,0.1,24,0.3,8.1,⋯,4.6,13.2,4.38,138,1.030,7,8.4,40,7.52,NA
6,0,F,CSRU,15,2,0.2,25,1.2,8.7,⋯,6.0,NA,3.75,142,1.016,25,23.9,56,7.43,206
7,1,M,MICU,19,3,0.0,33,0.3,8.9,⋯,4.6,15.0,5.01,147,1.030,34,21.0,56,7.36,365
8,0,M,CSRU,21,7,0.3,32,0.8,9.3,⋯,6.0,14.6,NA,139,1.015,31,16.8,63,6.00,518


Id,GENDER,FIRST_CAREUNIT,AnionGap,BaseExcess,Basophils,Bicarbonate,BilirubinTotal,CalciumTotal,CalculatedTotalCO2,⋯,Potassium,RDW,RedBloodCells,Sodium,SpecificGravity,UreaNitrogen,WhiteBloodCells,pCO2,pH,pO2
<int>,<chr>,<chr>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,⋯,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>
1,F,TSICU,16,0,0.0,32,1.1,9.4,23,⋯,3.8,12.8,5.04,143,1.048,NA,16.0,23,7.36,117
5,F,NICU,18,0,0.0,34,0.7,10.7,30,⋯,4.1,NA,5.12,152,1.017,NA,12.0,43,7.42,176
14,M,CSRU,16,11,0.8,33,0.3,8.7,39,⋯,6.5,21.7,4.10,NA,1.030,18,11.1,57,7.52,408
19,F,NICU,29,-1,0.9,27,NA,10.0,27,⋯,NA,21.8,2.88,143,1.015,142,24.0,103,7.34,102
25,F,CCU,20,5,0.8,NA,0.5,8.9,29,⋯,4.9,20.2,3.89,148,1.030,16,9.4,NA,7.43,326
34,M,CSRU,15,5,NA,28,0.4,9.3,32,⋯,4.7,14.6,4.09,142,1.015,39,28.1,62,7.50,631


In [3]:
train.x <- train[,-c(1,2)]
train.y <- train[,'Category']
train.x$FIRST_CAREUNIT <- as.factor(as.numeric(factor(train.x$FIRST_CAREUNIT)))
train.x$GENDER <- as.factor(as.numeric(factor(train.x$GENDER)))
head(train.x)

GENDER,FIRST_CAREUNIT,AnionGap,BaseExcess,Basophils,Bicarbonate,BilirubinTotal,CalciumTotal,CalculatedTotalCO2,Chloride,⋯,Potassium,RDW,RedBloodCells,Sodium,SpecificGravity,UreaNitrogen,WhiteBloodCells,pCO2,pH,pO2
<fct>,<fct>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>
2,2,14,2,0.0,35,0.4,9.0,26,121,⋯,5.9,14.2,4.21,152,1.017,25,34.5,52,7.46,278
1,2,12,0,0.4,28,0.7,9.1,25,111,⋯,5.4,14.4,3.38,142,1.048,24,14.2,35,5.00,194
2,2,12,3,0.1,24,0.3,8.1,30,107,⋯,4.6,13.2,4.38,138,1.030,7,8.4,40,7.52,NA
1,2,15,2,0.2,25,1.2,8.7,28,107,⋯,6.0,NA,3.75,142,1.016,25,23.9,56,7.43,206
2,3,19,3,0.0,33,0.3,8.9,28,110,⋯,4.6,15.0,5.01,147,1.030,34,21.0,56,7.36,365
2,2,21,7,0.3,32,0.8,9.3,33,106,⋯,6.0,14.6,NA,139,1.015,31,16.8,63,6.00,518


In [4]:
##feature pre-processing##
#checking variance of the features
nzv <- nearZeroVar(train.x, saveMetrics= TRUE)
#dropping zero variance features
#no zero var features
#nzv <- nearZeroVar(train.x)
        #nzv
#train_all.filt <- train_all[,-nzv] #3019 x 341

##looking for correlated variables
#descrCor <- cor(train.x)
    #summary(descrCor[upper.tri(descrCor)])
    #hist(descrCor[upper.tri(descrCor)])

#dropping highly correlate variables
#highlyCorDescr <- findCorrelation(descrCor, cutoff = .75) #accounting for install_id 
#no highly correlated variables
#train_all.clean <- train_all.filt[,-highlyCorDescr] #3019 x 1293

#center and scaling
pp_df <- preProcess(train.x, 
                     method = c("center", "scale",  "bagImpute", "YeoJohnson")) #knn for missing values

transformed.train <- predict(pp_df, newdata = train.x)


Building Model

In [5]:
#setting up cross validation
fitControl <- trainControl(## 10-fold CV
                           method = "repeatedcv",
                           number = 5,
                           ## repeated ten times
                           repeats =5 )

gbm <- train(x = transformed.train, y=factor(train.y$Category), 
                 method = 'gbm', 
                 trControl = fitControl)
                 ## This last option is actually one
                 ## for gbm() that passes through
                 #verbose = FALSE)

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        1.3368            -nan     0.1000    0.0063
     2        1.3269            -nan     0.1000    0.0050
     3        1.3177            -nan     0.1000    0.0043
     4        1.3089            -nan     0.1000    0.0043
     5        1.3019            -nan     0.1000    0.0034
     6        1.2953            -nan     0.1000    0.0031
     7        1.2898            -nan     0.1000    0.0021
     8        1.2841            -nan     0.1000    0.0026
     9        1.2788            -nan     0.1000    0.0025
    10        1.2739            -nan     0.1000    0.0023
    20        1.2385            -nan     0.1000    0.0013
    40        1.1990            -nan     0.1000    0.0006
    60        1.1764            -nan     0.1000    0.0004
    80        1.1614            -nan     0.1000    0.0002
   100        1.1510            -nan     0.1000    0.0001
   120        1.1423            -nan     0.1000    0.0001
   140        

In [6]:
#transforming test .

test$FIRST_CAREUNIT <- as.factor(as.numeric(factor(test$FIRST_CAREUNIT)))
test$GENDER <- as.factor(as.numeric(factor(test$GENDER)))
transformed.test <- predict(pp_df, newdata = test)

predictions <- predict(gbm, newdata = transformed.test)
#putting predictions back with installationid
pred_df <- cbind.data.frame(test$Id, predictions)
colnames(pred_df) <- c('Id', 'Category')
write.csv(pred_df, 'submission.csv', row.names = FALSE)